<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/-/blob/main/%E3%80%90note%E5%85%AC%E9%96%8B%E3%80%91%E7%84%A1%E6%96%99%E3%81%A7%E3%81%A4%E3%81%8B%E3%81%88%E3%82%8B%E3%81%AB%E3%82%83%E3%82%93%E3%81%93%E9%A2%A8%E9%8C%B2%E9%9F%B3%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
無料でつかえるにゃんこ録音ツールニ

In [2]:
from IPython.display import display, HTML
import base64
import io
import os

# 録音用のダイアログを表示する関数（にゃんこ風）
def record_audio_nyan():
    display(HTML('''
        <div style="font-family: 'Comic Sans MS', cursive; font-size: 18px; margin-bottom: 10px;">
            にゃんにゃん🎤 録音するにゃ！ボタン押してにゃ！
        </div>
        <button id="startBtn" onclick="startRecording()" style="font-size:16px;">🐾 録音開始にゃ！</button>
        <button id="stopBtn" onclick="stopRecording()" disabled style="font-size:16px;">🐾 録音やめるにゃ！</button>
        <audio id="audio" controls style="display: block; margin-top: 10px;"></audio>
        <div id="messageNyan" style="font-family: 'Comic Sans MS', cursive; font-size: 18px; margin-top: 10px;"></div>

        <script>
            let mediaRecorder;
            let audioChunks = [];
            let audioElement = document.getElementById('audio');
            let startBtn = document.getElementById('startBtn');
            let stopBtn = document.getElementById('stopBtn');
            let messageNyan = document.getElementById('messageNyan');

            async function startRecording() {
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream);
                mediaRecorder.start();
                mediaRecorder.ondataavailable = event => {
                    audioChunks.push(event.data);
                };
                startBtn.disabled = true;
                stopBtn.disabled = false;
                messageNyan.innerText = "にゃんにゃん🎶 録音中にゃ！";
            }

            async function stopRecording() {
                mediaRecorder.stop();
                mediaRecorder.onstop = async () => {
                    const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
                    const reader = new FileReader();
                    reader.readAsDataURL(audioBlob);
                    reader.onloadend = function() {
                        const base64data = reader.result.split(',')[1];
                        google.colab.kernel.invokeFunction('recordedAudio', [base64data], {});
                        audioElement.src = URL.createObjectURL(audioBlob);
                        messageNyan.innerText = "にゃーん！録音完了だにゃ！🐱✨";
                    };
                    audioChunks = [];
                };
                startBtn.disabled = false;
                stopBtn.disabled = true;
            }
        </script>
    '''))

# Python側で受け取る関数は同じ！
from google.colab import output

audio_data = None

def handle_recorded_audio(base64_audio):
    global audio_data
    audio_data = base64_audio

    audio_bytes = base64.b64decode(base64_audio)
    with open('tmp.wav', 'wb') as f:
        f.write(audio_bytes)

    os.system("ffmpeg -y -i tmp.wav -acodec pcm_s16le -ar 44100 recorded_audio.wav")

# コールバック登録
output.register_callback('recordedAudio', handle_recorded_audio)

# 呼び出し
record_audio_nyan()
